In [30]:
# Dependencies
import pandas as pd

In [31]:
# Import files using pandas
items_df = pd.read_csv("generated_data\items_complete.csv")
players_df = pd.read_csv("generated_data\players_complete.csv")
purchase_df = pd.read_csv("generated_data\purchase_data_3.csv")

In [32]:
# Get the total number of players with .count(), using SN column b/c its short
player_count = players_df["SN"].count()

In [33]:
# Get the total count of unique items with len() since .unique() returns a list
unique_item_count = len(items_df["Item Name"].unique())

In [34]:
# Get the average purchase price by dividing the sum of the price column by total number of items
average_purchase_price = purchase_df["Price"].sum() / len(purchase_df["Price"])

In [42]:
# Get the total number of purchases
total_purchases = purchase_df["Price"].count()

78


In [57]:
# Get total revenue by taking price from purchase and subtracting price from item
# join table -> do more stuff
#purchase_items_joined_df = pd.merge(purchase_df, items_df, on="Item ID")
#purchase_items_joined_df = purchase_items_joined_df.rename(columns={"Item Name_x": "Item Name(Purchase)",
#                                                                   "Price_x": "Purchase Price",
#                                                                   "Item Name_y": "Item Name(Stock)",
#                                                                   "Price_y": "MSRP"})
#purchase_items_joined_df["Revenue"] = purchase_items_joined_df["Purchase Price"] - purchase_items_joined_df["MSRP"]
#purchase_items_joined_df.head() FFFFFFFF I SHOULD'VE CHECKED THE GENERATOR CODE TO SEE IF THE PRICE WAS DIFFERENT
# OK actually just add up the revenue from the purchase_df
total_revenue = purchase_df["Price"].sum()

228.0999999999999


In [77]:
# Get % and # of male players
number_male = len(players_df.loc[players_df["Gender"] == "Male",:])

percent_male = (number_male / player_count)
percent_male = "{:.2%}".format(percent_male)
percent_male

'82.03%'

In [80]:
# Get % and # of female players
number_female = len(players_df.loc[players_df["Gender"] == "Female",:])

percent_female = (number_female / player_count)
percent_female = "{:.2%}".format(percent_female)
percent_female

'16.08%'

In [86]:
# Get % and # of other / non-disclosed players
number_other = len(players_df.loc[players_df["Gender"] == "Other / Non-Disclosed",:])

percent_other = (number_other / player_count)
percent_other = "{:.2%}".format(percent_other)
percent_other

'1.89%'

In [90]:
# Get purchase count broken by gender
male_purchase = len(purchase_df.loc[purchase_df["Gender"] == "Male",:])
female_purchase = len(purchase_df.loc[purchase_df["Gender"] == "Female",:])
other_purchase = len(purchase_df.loc[purchase_df["Gender"] == "Other / Non-Disclosed",:])

1

In [102]:
# Get average purchase price broken by gender
male_average_price = purchase_df.loc[purchase_df["Gender"] == "Male",:]
male_average_price = male_average_price["Price"].sum() / len(male_average_price["Price"])
male_average_price

female_average_price = purchase_df.loc[purchase_df["Gender"] == "Female",:]
female_average_price = female_average_price["Price"].sum() / len(female_average_price["Price"])
female_average_price

other_average_price = purchase_df.loc[purchase_df["Gender"] == "Other / Non-Disclosed",:]
other_average_price = other_average_price["Price"].sum() / len(other_average_price["Price"])
other_average_price

2.12

In [ ]:
# Get total purchase value broken by gender

In [91]:
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Iloni35,20,Male,93,Apocalyptic Battlescythe,4.49
1,1,Aidaira26,21,Male,12,Dawne,3.36
2,2,Irim47,17,Male,5,Putrid Fan,2.63
3,3,Irith83,17,Male,123,Twilight's Carver,2.55
4,4,Philodil43,22,Male,154,Feral Katana,4.11


In [44]:
items_df.head(1)

,Item ID,Item Name,Price
0,0,Splinter,1.89


In [79]:
players_df.head(10)

,Player ID,SN,Age,Gender
0,0,Marughi89,21,Male
1,1,Lirtedy26,40,Male
2,2,Chamistast30,7,Male
3,3,Lisirra25,24,Male
4,4,Lirtim36,23,Male
5,5,Undimsya85,17,Female
6,6,Lirtassa77,20,Female
7,7,Mindirra92,23,Male
8,8,Undirrasta89,23,Male
9,9,Iskjaskst81,15,Male


In [ ]:
# Things to do
#PLAYER COUNT
# - Total Number of Players
#PURCHASING ANALYSIS(TOTAL)
# - Number of Unique Items
# - Average Purchase Price
# - Total Number of Purchases
# - Total Revenue
#GENDER DEMOGRAPHICS
# - Percentage and Count of Male Players
# - Percentage and Count of Female Players
# - Percentage and Count of Other / Non-Disclosed
#PURCHASING ANALYSIS (GENDER)
#>broken by gender
# - Purchase Count
# - Average Purchase Price
# - Total Purchase Value
#AGE DEMOGRAPHICS
#TOP SPENDERS
#MOST POPULAR ITEMS
#MOST PROFITABLE ITEMS